In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import os.path

if not os.path.isfile('models.py'):
    from google.colab import drive
    drive.mount('/content/drive')

!pip install pandas_summary
#!pip install --upgrade pandas

Mounted at /content/drive


https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [ ]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

In [ ]:
if not os.path.isfile('models.py'):
    df = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/train_normalized_data2.fth')
    df_test = pd.read_feather('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/test_normalized_data2.fth')
else:
    df = pd.read_feather('train_normalized_data2.fth')
    df_test = pd.read_feather('test_normalized_data2.fth')

In [ ]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', #'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
#            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', #'SchoolHoliday_bw']

cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks',           'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

    

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [ ]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

contin_vars = ['CompetitionDistance', 
'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']   
# contin_vars = []

In [ ]:
y_out_columns = ['Sales']

In [ ]:
# split_type = 'random'
split_type = 'no_split'
#split_type = 'last_week'

In [ ]:
# Esta es para entrenar con todo
if split_type == 'no_split':
    df_train = df
elif split_type == 'last_week':
    # Esto divide en train y val, como venimos haciendo
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
    df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
elif split_type == 'random':
    # Splitting aleatorio
    np.random.seed(42)
    indexes = np.arange(len(df))
    np.random.shuffle(indexes)
    N = len(df)//5
    df_train = df[N:]
    df_val = df[:N]
    print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')


#df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
#df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
#print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

In [ ]:
X_train = df_train[cat_vars + contin_vars]
if split_type != 'no_split':
    X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [ ]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    if split_type != 'no_split':
        y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    if split_type != 'no_split':
        y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation

In [ ]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'


In [ ]:
a = np.hsplit(X_train, X_train.shape[1])
len(a)

40

In [ ]:
X_train.shape

(844338, 40)

In [ ]:
X_train.shape[1]

40

In [ ]:
# XGBoost no admite features categoricos, por eso hay que hacer un pre-procesamiento para poder incluirlos

# El uso de embeddings, en este caso utilizados los embeddings generado MLP de 06-full-model
if categoricals_processing == 'use_embeddings':
    if not os.path.isfile('models.py'):
      embeddings_model = load_model('/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/Competencias/10-1-Rossman/embeddings_model2.hdf5') #Cargo los embeddin hasta realizar el concatenate
    else:
      embeddings_model = load_model('embeddings_model.hdf5') #Cargo los embeddin hasta realizar el concatenate
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1) #pasa de un matriz de (814150, 40) de DataSet a un array de 40 listas
    if split_type != 'no_split':
      X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

In [ ]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    if split_type != 'no_split':
      X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

## XGBoost

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [ ]:
n_estimators=4000
learning_rate=0.25
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=12, max_depth=8, gamma=0.005)
model

In [ ]:
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'rmse', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
score = model.score(X_val, y_val)

In [ ]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [ ]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [ ]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [ ]:
print(score, train_RMSE, val_RMSE)

# HYPER-SEARCH

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
fit_params={#"early_stopping_rounds":100, 
            #"eval_metric" : 'rmse', 
            #"eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }
#Parametros para realizar la busqueda en BayesSearchCV
param_bayes = {
        'max_depth': Integer(1,31),
        'learning_rate': Real(np.exp(-6), np.exp(-1), prior='log-uniform'),
        'reg_lambda': Real(np.exp(-8), np.exp(-3), prior='log-uniform'),
        'gamma': Real(np.exp(-8), np.exp(-3), prior='log-uniform'),
        'min_child_samples': Integer(1, 20),
        'min_child_weight': Real(np.exp(-8), np.exp(-1))  }
opt = BayesSearchCV(XGBRegressor(n_estimators=4000, n_jobs=8, objective='reg:squarederror'), param_bayes, n_iter=32,random_state=0, n_jobs=-1, verbose=1, fit_params=fit_params)


fitBayesSearchCV = opt.fit(X_train, y_train.reshape(-1))

In [ ]:
#summarize results
print("Best: %f using %s" % (fitBayesSearchCV.best_score_, fitBayesSearchCV.best_params_))
means = fitBayesSearchCV.cv_results_['mean_test_score']
stds = fitBayesSearchCV.cv_results_['std_test_score']
params = fitBayesSearchCV.cv_results_['params']
print('--------------------------------------------------------------')
for mean, stdev, param in zip(means, stds, params):    
    print("%f (%f) with: %r" % (mean, stdev, param))

# Baseline

In [ ]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [ ]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [ ]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test[['Store', 'Sales']].head(10)

In [ ]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

In [ ]:
sample_csv['Sales'] = df_test['Sales']

In [ ]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [ ]:
sample_csv.head()

# Sumbit a la competición

In [ ]:
sample_csv = pd.read_csv(r'./dataset/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)

In [ ]:
#SOCRE 0.24541

In [ ]:
cat_vars